In [1]:
import pandas as pd
import glob

C:\Users\amani\AppData\Local\Temp\ipykernel_24932\329695546.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Clean Packet Capture

In [6]:
# List of all csv in a dir
csv_files = glob.glob('captures1_v2/clean/*.csv')

combined_df = pd.DataFrame()

# append to combined dataframe
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    combined_df = pd.concat([combined_df, df])

combined_df['AttackName'] = 'Clean'

combined_df.head()

,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,AttackName
0,1,0.000000,172.27.224.70,172.27.224.250,TCP,60,49499 > 502 [ACK] Seq=1 Ack=1 Win=65051 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.535046e+09,0.000000,0.000000,Clean
1,2,0.020940,HewlettP_8e:40:b3,Spanning-tree-(for-bridges)_00,STP,64,RST. Root = 32768/0/00:18:6e:d7:8a:c0 Cost = ...,d0:7e:28:8e:40:b3,HewlettP_8e:40:b3,Hewlett Packard,...,NaN,NaN,Spanning-tree-(for-bridges)_00,NaN,NaN,STP,1.535046e+09,0.020940,0.020940,Clean
2,3,0.094309,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.535046e+09,0.094309,0.073369,Clean
3,4,0.097427,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,"VMware, Inc.",49499.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.535046e+09,0.097427,0.003118,Clean
4,5,0.311972,172.27.224.70,172.27.224.250,TCP,60,49499 > 502 [ACK] Seq=13 Ack=32 Win=65020 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.535046e+09,0.311972,0.214545,Clean
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427837,427838,21598.650849,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,"VMware, Inc.",49205.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.536452e+09,21598.650849,0.010907,Clean
427838,427839,21598.857675,172.27.224.70,172.27.224.250,TCP,60,49205 > 502 [ACK] Seq=817261 Ack=2111256 Win...,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.536452e+09,21598.857675,0.206826,Clean
427839,427840,21598.952198,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.536452e+09,21598.952198,0.094523,Clean
427840,427841,21598.960188,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,"VMware, Inc.",49205.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.536452e+09,21598.960188,0.007990,Clean


# Man in the Middle Attack

In [7]:
csv_files = glob.glob('captures1_v2/mitm/*.csv')

combined_df = pd.DataFrame()

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    combined_df = pd.concat([combined_df, df])

combined_df['AttackName'] = 'mitm'

combined_df.head()

,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,AttackName
0,1,0.000000,172.27.224.70,172.27.224.250,TCP,60,49499 > 502 [ACK] Seq=1 Ack=1 Win=65237 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.535061e+09,0.000000,0.000000,mitm
1,2,0.094173,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.535061e+09,0.094173,0.094173,mitm
2,3,0.102293,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,"VMware, Inc.",49499.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.535061e+09,0.102293,0.008120,mitm
3,4,0.312027,172.27.224.70,172.27.224.250,TCP,60,49499 > 502 [ACK] Seq=13 Ack=32 Win=65206 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.535061e+09,0.312027,0.209734,mitm
4,5,0.406373,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.535061e+09,0.406373,0.094346,mitm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419372,419373,21598.584789,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.535135e+09,21598.584789,0.059367,mitm
419373,419374,21598.590545,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,"VMware, Inc.",49499.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.535135e+09,21598.590545,0.005756,mitm
419374,419375,21598.802586,172.27.224.70,172.27.224.250,TCP,60,49499 > 502 [ACK] Seq=805585 Ack=2081093 Win...,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.535135e+09,21598.802586,0.212041,mitm
419375,419376,21598.896686,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.535135e+09,21598.896686,0.094100,mitm


# modbusQuery2Flooding

In [17]:
csv_files = glob.glob('captures1_v2/modbusQuery2Flooding/*.csv')

combined_df = pd.DataFrame()

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file, on_bad_lines='skip')
        combined_df = pd.concat([combined_df, df])
    except Exception as e:
        print(f"Failed to read {csv_file}: {e}")

combined_df['AttackName'] = 'modbusQuery2Flooding'

combined_df.head()

Failed to read captures1_v2/modbusQuery2Flooding\eth2dump-modbusQuery2Flooding30m-1h_1.csv: 'utf-8' codec can't decode byte 0x86 in position 246153: invalid start byte


,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,AttackName
0,1,0.000000,172.27.224.70,172.27.224.250,TCP,60,49179 > 502 [ACK] Seq=1 Ack=1 Win=64927 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529551e+09,0.000000,0.000000,modbusQuery2Flooding
1,2,0.057897,172.27.224.251,172.27.224.250,TCP,60,"51111 > 502 [FIN, ACK] Seq=1 Ack=1 Win=2036 ...",48:5b:39:64:40:79,ASUSTekC_64:40:79,ASUSTek COMPUTER INC.,...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529551e+09,0.057897,0.057897,modbusQuery2Flooding
2,3,0.067569,172.27.224.250,172.27.224.251,TCP,60,502 > 51111 [ACK] Seq=1 Ack=2 Win=8712 Len=0,00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,ASUSTek COMPUTER INC.,51111.0,ASUSTekC_64:40:79,Not set,Set,TCP,1.529551e+09,0.067569,0.009672,modbusQuery2Flooding
3,4,0.068851,172.27.224.250,172.27.224.251,TCP,60,"502 > 51111 [FIN, ACK] Seq=1 Ack=2 Win=8712 ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,ASUSTek COMPUTER INC.,51111.0,ASUSTekC_64:40:79,Not set,Set,TCP,1.529551e+09,0.068851,0.001282,modbusQuery2Flooding
4,5,0.068872,172.27.224.251,172.27.224.250,TCP,60,51111 > 502 [ACK] Seq=2 Ack=2 Win=2036 Len=0,48:5b:39:64:40:79,ASUSTekC_64:40:79,ASUSTek COMPUTER INC.,...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529551e+09,0.068872,0.000021,modbusQuery2Flooding


# modbusQueryFlooding

In [20]:
csv_files = glob.glob('captures1_v2/modbusQueryFlooding/*.csv')

combined_df = pd.DataFrame()

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file, on_bad_lines='skip')
        combined_df = pd.concat([combined_df, df])
    except Exception as e:
        print(f"Failed to read {csv_file}: {e}")

combined_df['AttackName'] = 'modbusQueryFlooding'

combined_df.head()

Failed to read captures1_v2/modbusQueryFlooding\eth2dump-modbusQueryFlooding15m-6h_1.csv: Unable to allocate 36.6 MiB for an array with shape (1, 4803413) and data type object
Failed to read captures1_v2/modbusQueryFlooding\eth2dump-modbusQueryFlooding30m-1h_1.csv: Unable to allocate 165. MiB for an array with shape (4, 5412081) and data type object
Failed to read captures1_v2/modbusQueryFlooding\eth2dump-modbusQueryFlooding30m-6h_1.csv: Unable to allocate 217. MiB for an array with shape (5, 5686258) and data type object
Failed to read captures1_v2/modbusQueryFlooding\eth2dump-modbusQueryFlooding5m-0,5h_1.csv: Unable to allocate 159. MiB for an array with shape (4, 5215622) and data type object
Failed to read captures1_v2/modbusQueryFlooding\eth2dump-modbusQueryFlooding5m-1h_1.csv: Unable to allocate 161. MiB for an array with shape (4, 5268951) and data type object
Failed to read captures1_v2/modbusQueryFlooding\eth2dump-modbusQueryFlooding5m-6h_1.csv: Unable to allocate 214. MiB for

,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,TimeDelta,RelativeTime,AttackName
0,1,0.000000,172.27.224.70,172.27.224.250,TCP,60,49179 > 502 [ACK] Seq=1 Ack=1 Win=65175 Len=0,VMware_9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemecanique Electrique,502.0,Telemecaniqu_09:51:3b,Not set,Set,TCP,1.529547e+09,Not set,0.000000,modbusQueryFlooding
1,2,0.095448,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",VMware_9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemecanique Electrique,502.0,Telemecaniqu_09:51:3b,Not set,Set,Modbus/TCP,1.529547e+09,Not set,0.095448,modbusQueryFlooding
2,3,0.102847,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",Telemecaniqu_09:51:3b,Telemecaniqu_09:51:3b,Telemecanique Electrique,...,"VMware, Inc.",49179.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.529547e+09,Not set,0.102847,modbusQueryFlooding
3,4,0.273109,HewlettPacka_8e:40:b3,Spanning-tree-(for-bridges)_00,STP,64,RST. Root = 32768/0/00:18:6e:d7:8a:c0 Cost = ...,HewlettPacka_8e:40:b3,HewlettPacka_8e:40:b3,Hewlett Packard,...,IEEE 802.1 Chair,NaN,Spanning-tree-(for-bridges)_00,NaN,NaN,STP,1.529547e+09,NaN,0.273109,modbusQueryFlooding
4,5,0.311994,172.27.224.70,172.27.224.250,TCP,60,49179 > 502 [ACK] Seq=13 Ack=32 Win=65144 Len=0,VMware_9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemecanique Electrique,502.0,Telemecaniqu_09:51:3b,Not set,Set,TCP,1.529547e+09,Not set,0.311994,modbusQueryFlooding


# TcpSYNFlood

In [14]:
csv_files = glob.glob('captures1_v2/tcpSYNFloodDDoS/*.csv')

# Create an empty dataframe to store the combined data
combined_df = pd.DataFrame()

# Loop through each CSV file and append its contents to the combined dataframe
for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file, on_bad_lines='skip')
        combined_df = pd.concat([combined_df, df])
    except Exception as e:
        print(f"Failed to read {csv_file}: {e}")

combined_df['AttackName'] = 'tcpSYNFloodDDoS'

combined_df.head()

Failed to read captures1_v2/tcpSYNFloodDDoS\eth2dump-tcpSYNFloodDDoS30m-6h_1.csv: 'utf-8' codec can't decode byte 0x86 in position 261916: invalid start byte


,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,AttackName
0,1,0.000000,172.27.224.70,172.27.224.250,TCP,60,49179 > 502 [ACK] Seq=1 Ack=1 Win=64865 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529543e+09,0.000000,0.000000,tcpSYNFloodDDoS
1,2,0.111408,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.529543e+09,0.111408,0.111408,tcpSYNFloodDDoS
2,3,0.120144,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,"VMware, Inc.",49179.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.529543e+09,0.120144,0.008736,tcpSYNFloodDDoS
3,4,0.238885,HewlettP_8e:40:b3,Spanning-tree-(for-bridges)_00,STP,64,RST. Root = 32768/0/00:18:6e:d7:8a:c0 Cost = ...,d0:7e:28:8e:40:b3,HewlettP_8e:40:b3,Hewlett Packard,...,NaN,NaN,Spanning-tree-(for-bridges)_00,NaN,NaN,STP,1.529543e+09,0.238885,0.118741,tcpSYNFloodDDoS
4,5,0.327575,172.27.224.70,172.27.224.250,TCP,60,49179 > 502 [ACK] Seq=13 Ack=32 Win=65392 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529543e+09,0.327575,0.088690,tcpSYNFloodDDoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406650,406651,21599.029599,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.527250e+09,21599.029599,0.095347,tcpSYNFloodDDoS
406651,406652,21599.036850,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,"VMware, Inc.",50891.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.527250e+09,21599.036850,0.007251,tcpSYNFloodDDoS
406652,406653,21599.246255,172.27.224.70,172.27.224.250,TCP,60,50891 > 502 [ACK] Seq=746965 Ack=1929658 Win...,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.527250e+09,21599.246255,0.209405,tcpSYNFloodDDoS
406653,406654,21599.342068,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.527250e+09,21599.342068,0.095813,tcpSYNFloodDDoS


# pingFloodDDoS

In [19]:
csv_files = glob.glob('captures1_v2/pingFloodDDoS/*.csv')

combined_df = pd.DataFrame()

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file, on_bad_lines='skip')
        combined_df = pd.concat([combined_df, df])
    except Exception as e:
        print(f"Failed to read {csv_file}: {e}")

combined_df['AttackName'] = 'pingFloodDDoS'

combined_df.head()

,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,AttackName
0,1,0.000000,172.27.224.70,172.27.224.250,TCP,60,49179 > 502 [ACK] Seq=1 Ack=1 Win=65299 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529539e+09,0.000000,0.000000,pingFloodDDoS
1,2,0.142182,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.529539e+09,0.142182,0.142182,pingFloodDDoS
2,3,0.145178,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,"VMware, Inc.",49179.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.529539e+09,0.145178,0.002996,pingFloodDDoS
3,4,0.358785,172.27.224.70,172.27.224.250,TCP,60,49179 > 502 [ACK] Seq=13 Ack=32 Win=65268 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529539e+09,0.358785,0.213607,pingFloodDDoS
4,5,0.380756,HewlettP_8e:40:b3,Spanning-tree-(for-bridges)_00,STP,64,RST. Root = 32768/0/00:18:6e:d7:8a:c0 Cost = ...,d0:7e:28:8e:40:b3,HewlettP_8e:40:b3,Hewlett Packard,...,NaN,NaN,Spanning-tree-(for-bridges)_00,NaN,NaN,STP,1.529539e+09,0.380756,0.021971,pingFloodDDoS
